In [ ]:
#!pip install pyspark

In [1]:
from random import random
import timeit

In [2]:
from pyspark.sql import SparkSession
MAX_MEMORY = "8g"
spark = SparkSession \
    .builder \
    .appName("SparkdADDemo") \
    .config("spark.executor.memory", MAX_MEMORY) \
    .config("spark.driver.memory", MAX_MEMORY) \
    .config("spark.ui.port", "4050") \
    .master("local[*]") \
    .getOrCreate()

In [3]:
spark

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
value="$(cat /content/drive/MyDrive/BU\ MF810\ 2025/ngrok)"

!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xvzf ngrok-v3-stable-linux-amd64.tgz

!./ngrok config add-authtoken $value

get_ipython().system_raw('./ngrok http 4050 &')
!sleep 5

--2025-04-10 19:12:33--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 35.71.179.82, 99.83.220.108, 75.2.60.68, ...
Connecting to bin.equinox.io (bin.equinox.io)|35.71.179.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9372184 (8.9M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-linux-amd64.tgz’

ngrok-v3-stable-lin 100%[===================>]   8.94M  11.5MB/s    in 0.8s    

2025-04-10 19:12:34 (11.5 MB/s) - ‘ngrok-v3-stable-linux-amd64.tgz’ saved [9372184/9372184]

ngrok
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [6]:
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[{"name":"command_line","ID":"366b5393b5930f502a990eb2810d3a3f","uri":"/api/tunnels/command_line","public_url":"https://b2d6-34-68-152-240.ngrok-free.app","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}


In [7]:
num_samples = 10000000

In [8]:
manyNumbers = spark \
    .sparkContext \
    .parallelize(range(0, num_samples))
manyNumbers.take(5)

[0, 1, 2, 3, 4]

In [9]:
times5 = manyNumbers.map(lambda x: x*5)
times5.take(5)

[0, 5, 10, 15, 20]

In [10]:
split7 = manyNumbers.repartition(7)
split7.take(2)

[40, 41]

In [11]:
ds1 = spark.sparkContext.parallelize(range(0, num_samples))
ds2 = spark.sparkContext.parallelize(range(0, num_samples, 2))
ds3 = ds1.repartition(7)
ds4 = ds2.repartition(9)
ds5 = ds3.map(lambda x: x*5)

In [12]:
joined = ds5 + ds4
joined.sum()

274999970000000

In [13]:
spark.stop()